In [57]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.cluster import KMeans

DATASET

In [61]:
enfermedades = pd.read_csv('https://raw.githubusercontent.com/VictorD2/mainframes-server/main/data/heart%20disease%20classification%20dataset.csv',engine="python")
enfermedades

,id,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,0,63,male,3,145.0,233.0,1,0,150.0,0,2.3,0,0,1,yes
1,1,37,male,2,130.0,250.0,0,1,187.0,0,3.5,0,0,2,yes
2,2,41,female,1,130.0,204.0,0,0,172.0,0,1.4,2,0,2,yes
3,3,56,male,1,120.0,236.0,0,1,178.0,0,0.8,2,0,2,yes
4,4,57,female,0,NaN,354.0,0,1,163.0,1,0.6,2,0,2,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,298,57,female,0,140.0,241.0,0,1,123.0,1,0.2,1,0,3,no
299,299,45,male,3,110.0,264.0,0,1,132.0,0,1.2,1,0,3,no
300,300,68,male,0,144.0,193.0,1,1,141.0,0,3.4,1,2,3,no
301,301,57,male,0,NaN,131.0,0,1,115.0,1,1.2,1,1,3,no


DICCIONARIO DE DATOS

Diccionario de datos

age: Muestra la edad de la persona.

sex: Muestra el género del individuo usando el siguiente formato: 1 = masculino 0 = femenino

cp - Tipo de dolor torácico: Muestra el tipo de dolor torácico experimentado por el individuo utilizando el siguiente formato: 0 = angina típica 1 = angina atípica 2 = dolor no anginoso 3 = asintótico

trestbps- Presión arterial en reposo: muestra el valor de la presión arterial en reposo de una persona en mmHg (unidad). cualquier cosa por encima de 130-140 suele ser motivo de preocupación.

chol- Colesterol sérico: muestra el colesterol sérico en mg/dl (unidad)

fbs- Azúcar en sangre en ayunas: compara el valor de azúcar en sangre en ayunas de un individuo con 120 mg/dl. Si el nivel de azúcar en sangre en ayunas es > 120 mg/dl, entonces: 1 (verdadero) de lo contrario: 0 (falso) '>126' mg/dl indica diabetes

restecg- ECG en reposo: muestra los resultados electrocardiográficos en reposo 0 = normal 1 = tiene anomalías en la onda ST-T 2 = hipertrofia ventricular izquierda

thalach- Frecuencia cardíaca máxima alcanzada: muestra la frecuencia cardíaca máxima alcanzada por una persona.

exang- Angina inducida por el ejercicio: 1 = sí 0 = no

oldpeak- Depresión del ST inducida por el ejercicio en relación con el reposo: muestra el valor que es un número entero o flotante.

Pendiente: Pendiente del segmento ST del ejercicio máximo: 0 = pendiente ascendente: mejor frecuencia cardíaca con el ejercicio (poco común) 1 = plana: cambio mínimo (típico corazón sano) 2 = pendiente descendente: signos de corazón no saludable

ca- Número de vasos principales (0–3) coloreados por fluoroscopia: muestra el valor como entero o flotante.

thal: Muestra la talasemia: 1,3 = normal 6 = defecto fijo 7 = defecto reversible: no hay circulación sanguínea adecuada al hacer ejercicio

target : Muestra si el individuo sufre o no una enfermedad cardíaca : 1 = sí 0 = no

In [ ]:
enfermedades.info()

In [ ]:
def setGender(gender):
    return 1 if gender == "male" else 0

def setTarget(target):
    return 1 if target == "yes" else 0

MODIFICANDO LOS VALORES DE LAS COLUMNAS QUE NO SON NÚMEROS

In [ ]:
enfermedades["target"] = enfermedades.target.apply(setTarget)
enfermedades["sex"] = enfermedades.sex.apply(setGender)

In [ ]:
enfermedades

In [ ]:
enfermedades.isnull().sum()

In [ ]:
columnas = enfermedades.columns
# Conjunto de variables de entrada
X = enfermedades.iloc[:, 1:].values
# Conjunto de variables de salida
y = enfermedades.iloc[:, 0].values

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.tree import DecisionTreeRegressor 

# Conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
# Librerías necesarias
imputer = IterativeImputer(estimator=DecisionTreeRegressor(), max_iter=10, imputation_order="ascending", random_state=21)
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

In [ ]:
# Conversión de los conjuntos de entrenamiento y prueba a DataFrame
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [ ]:
# Concatenación de los conjuntos de entrenamiento y prueba en variables de entrada y salida
X = pd.concat([X_train, X_test], axis=0)
y = pd.concat([y_train, y_test], axis=0)

In [ ]:
# Concatenación de los conjuntos de entrada y salida a un conjunto global
data = pd.concat([y, X], axis=1)

# Le devolvemos los nombres a las columnas
data.columns = columnas

data.isnull().sum()

In [ ]:
enfermedades

In [ ]:
data = data.sort_values(by="id")

In [ ]:
data.to_csv('./data/DATOS.csv')